In [1]:
!pip install transformers==4.40.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

from transformers import logging
logging.set_verbosity_error()

import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

import sys
sys.path.append('/kaggle/input/chexagent-files/CheXagent')
from model_chexagent.chexagent import CheXagent

# ───────────────────────────────────────────────────
# 1) Load ground truth CSV
# ───────────────────────────────────────────────────
gt_df = pd.read_csv("/kaggle/input/chest-x-ray-ground-truth-labels/test_df/test_df.csv")
gt_df["ImageName"] = gt_df["ImageName"].apply(os.path.basename)
diseases = [col for col in gt_df.columns if col != "ImageName"]

# ───────────────────────────────────────────────────
# 2) Initialize CheXagent model
# ───────────────────────────────────────────────────
chexagent = CheXagent()

# ───────────────────────────────────────────────────
# 3) Batch inference & per-batch CSV + accuracy
# ───────────────────────────────────────────────────
batch_size = 100
num_batches = 10  # only do 10 batches

for batch_idx in range(num_batches):
    start = batch_idx * batch_size
    end   = start + batch_size
    batch_df = gt_df.iloc[start:end]
    if batch_df.empty:
        break  # no more images

    preds = []
    for _, row in tqdm(batch_df.iterrows(),
                       total=len(batch_df),
                       desc=f"Batch {batch_idx+1}/{num_batches}"):
        img_file = row["ImageName"]
        img_path = os.path.join(
            "/kaggle/input/chest-x-ray-images/test_data",
            img_file
        )
        row_pred = {"ImageName": img_file}
        for disease in diseases:
            response = chexagent.binary_disease_classification([img_path], disease)
            txt = response.strip().lower()
            # 1 if CheXagent says yes/present/true or mentions the disease name
            flag = int(
                txt.startswith(("yes", "present", "true")) or
                (disease.lower() in txt)
            )
            row_pred[disease] = flag
        preds.append(row_pred)

    # Build DataFrame & save CSV
    pred_df = pd.DataFrame(preds)
    out_csv = f"predictions_batch_{batch_idx+1}.csv"
    pred_df.to_csv(out_csv, index=False)
    print(f"Saved predictions for batch {batch_idx+1} → {out_csv}")

    # Compute per-batch accuracy
    merged = batch_df.merge(pred_df, on="ImageName", suffixes=("_gt","_pred"))
    # Flatten all labels to compute overall accuracy for the batch
    y_true = merged[[d + "_gt" for d in diseases]].values.flatten()
    y_pred = merged[[d + "_pred" for d in diseases]].values.flatten()
    acc = accuracy_score(y_true, y_pred)
    print(f"Batch {batch_idx+1} accuracy: {acc:.3f}\n")


tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenization_chexagent.py:   0%|          | 0.00/26.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.85k [00:00<?, ?B/s]

...

Saved predictions for batch 8 → predictions_batch_8.csv
Batch 8 accuracy: 0.751



Batch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

Saved predictions for batch 9 → predictions_batch_9.csv
Batch 9 accuracy: 0.765



Batch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

Saved predictions for batch 10 → predictions_batch_10.csv
Batch 10 accuracy: 0.735

